In [1]:
# Do Mobile-Clf-Ram and Mobile-Clf-Int-Memory
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    LabelEncoder,
    Normalizer,
)
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,VotingRegressor,BaggingRegressor,RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
np.random.seed(42)

In [2]:
data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
X = data.drop(['price_range'],axis=1)
y = data[['price_range']]

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [5]:
import wandb
project_name = 'Mobile-Price-Clf'

In [6]:
wandb.init(project=project_name,name='baseline')
model = GradientBoostingRegressor()
model.fit(X_train,y_train)
wandb.log({'Acc':model.score(X_train,y_train)})
wandb.log({'Val Acc':model.score(X_test,y_test)})

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [7]:
pres = [StandardScaler(),RobustScaler(),MinMaxScaler(),MaxAbsScaler(),Normalizer(),LabelEncoder()]

In [8]:
label_encoder = True

In [9]:
PROJECT_NAME = project_name

In [10]:
# for pre in pres:
#     X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)
#     if label_encoder is True:
#         le = LabelEncoder()
#         le.fit(y_train)
#         y_train = le.transform(y_train)
#     wandb.init(project=PROJECT_NAME,name=f'preproccessing-{pre}-labelencoder-{label_encoder}')
#     transformation = pre
#     transformation.fit(X_train)
#     X_train = transformation.transform(X_train)
#     model = GradientBoostingRegressor()
#     print('Fitting')
#     model.fit(X_train,y_train)
#     print('Scoring Train')
#     acc = model.score(X_train,y_train)
#     wandb.log({'Acc':acc})
#     print('Scoring Test')
#     acc = model.score(transformation.transform(X_test),y_test)
#     wandb.log({'Val Acc':acc})

In [11]:
del test_data['id']

In [12]:
transformation = StandardScaler()
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)
X_train = transformation.fit_transform(X_train)
X_test = transformation.transform(X_test)
test_data = transformation.transform(test_data)

In [13]:
models = [
    ['KNeighborsRegressor',KNeighborsRegressor],
    ['ExtraTreesRegressor',ExtraTreesRegressor],
    ['BaggingRegressor',BaggingRegressor],
    ['LogisticRegression',LogisticRegression],
    ['GaussianNB',GaussianNB],
    ['DecisionTreeRegressor',DecisionTreeRegressor],
    ['GradientBoostingRegressor',GradientBoostingRegressor],
    ['AdaBoostRegressor',AdaBoostRegressor],
    ['RandomForestRegressor',RandomForestRegressor],
    ['BaggingRegressor',BaggingRegressor],
    ['SVR',SVR],
]

In [14]:
# for model in models:
#     wandb.init(project=PROJECT_NAME,name=f'{model[0]}')
#     model = model[1]()
#     model.fit(X_train,y_train)
#     acc = model.score(X_train,y_train)
#     wandb.log({'Acc':acc})
#     print('Scoring Test')
#     acc = model.score(X_test,y_test)
#     wandb.log({'Val Acc':acc})

In [15]:
# wandb.init(project=project_name,name='baseline')
# model = RandomForestRegressor()
# model.fit(X_train,y_train)
# wandb.log({'Acc':model.score(X_train,y_train)})
# wandb.log({'Val Acc':model.score(X_test,y_test)})

In [16]:
from sklearn.model_selection import GridSearchCV

In [17]:
import pickle

In [19]:
wandb.init(project=PROJECT_NAME,name='Final')
model = RandomForestRegressor(bootstrap=True,criterion= 'mse',max_depth= 10,max_features='auto',n_estimators= 250,oob_score= False)
model.fit(X_train,y_train)
wandb.log({'Acc':model.score(X_train,y_train)})
wandb.log({'Val Acc':model.score(X_test,y_test)})

Acc,0.96384
_runtime,6
_timestamp,1628426194
_step,1
Val Acc,0.93382


Acc,▁
_runtime,▁▁
_timestamp,▁▁
_step,▁█
Val Acc,▁


wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/indika/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
pickle.dump(model,open('./model.pkl','wb'))

In [23]:
ids = pd.read_csv('./test.csv')['id']

In [24]:
preds = model.predict(test_data)

In [25]:
df = {'id':[],'pred':[]}

In [26]:
for _id,pred in zip(ids,preds):
    df['id'].append(_id)
    df['pred'].append(pred)

In [29]:
test_data_preds = pd.DataFrame(df)

In [31]:
test_data_preds.to_csv('./test_data_preds.csv',index=False)